In [156]:
#hide
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import altair as alt
from load_covid_data import load_individual_timeseries
%config InlineBackend.figure_format = 'retina'

chart_width = 1080
chart_height= 720

In [157]:
Since reaching at least 1 death per million

> Tip: Click (Shift+ for multiple) on countries in the legend to filter the visualization.

SyntaxError: invalid syntax (<ipython-input-157-26a600a95de1>, line 1)

In [158]:
#hide 
data = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv", 
                   error_bad_lines=False)
data = data.drop(columns=["Lat", "Long"])
data = data.melt(id_vars= ["Province/State", "Country/Region"])
data = pd.DataFrame(data.groupby(['Country/Region', "variable"]).sum())
data.reset_index(inplace=True)  
data = data.rename(columns={"Country/Region": "location", "variable": "date", "value": "total_cases"})
data['date'] =pd.to_datetime(data.date)
data = data.sort_values(by = "date")
data.loc[data.location == "US","location"] = "United States"
data.loc[data.location == "Korea, South","location"] = "South Korea"
countries=pd.Series(data.location.unique())


In [159]:
#hide
# Population data
data_pop=pd.read_csv("data_files/world_population.csv")[["country","population"]]
data_pop=data_pop[data_pop.country.isin(countries)]
# To list the countries who's population is not found in world bank
# pd.set_option('display.max_rows', None)
# countries=countries[~countries.isin(data_pop.country)]
countries=countries[countries.isin(data_pop.country)]


display(data_pop.country.unique().size , countries.size)
# (countries.isin(data_pop.country))[countries == False]

175

175

In [160]:
# data_pc=data_pc[data_pc.location == 'China']
# data_pc.total_cases.max()
# data_pop=data_pop.apply(lambda x: [(x/1000000)], 1)
# data_pop=data_pop[data_pop.country.isin(countries)]
# data_pop=data_pop[data_pop.population > 0]

data_pc=data.copy()
data_pc=data_pc[data_pc.location.isin(countries)]

# pd.set_option('display.max_rows', 10)
# data_pc[data_pc.total_cases==float('nan') | float('inf')]
# data_pc

In [161]:
for i in countries:
    data_pc.loc[data_pc.location == i,"total_cases"] = data_pc.loc[data_pc.location == i,"total_cases"]/(data_pop.loc[data_pop.country == i, "population"].astype(float).to_list()[0]/1000000)

data_countries_pc=[]

ranked_data=pd.DataFrame(columns=['country','total_cases_per_milliion_pop'])#country, total_cases_per_milliion_pop
# ranked_data.append({'a','b'})
for i in countries:
    filter_country = data_pc["location"]== i
    temp=data_pc[filter_country]
    # display(temp)
    if temp['total_cases'].sum() > 0 :
        ranked_data=ranked_data.append([{'country':i,'total_cases_per_milliion_pop':temp['total_cases'].tail(1).to_list()[0]}])

ranked_data=ranked_data.sort_values(by='total_cases_per_milliion_pop',ignore_index=True,ascending=False)
countries=ranked_data['country']
for i in countries:
    filter_country = data_pc["location"]== i
    temp=data_pc[filter_country]
    data_countries_pc.append(data_pc[filter_country])
pd.set_option('display.max_rows', None)
display(ranked_data)


,country,total_cases_per_milliion_pop
0,San Marino,1208.334561
1,Belgium,763.010151
2,Andorra,634.181065
3,Spain,579.705458
4,Italy,514.473380
5,United Kingdom,489.997858
6,France,414.823710
7,Sweden,342.599189
8,Netherlands,325.710049
9,Ireland,303.172312


In [162]:
pd.set_option('display.max_rows', 10)
len(data_countries_pc)

151

In [163]:
for i in range(0,len(countries)):
    data_countries_pc[i]=data_countries_pc[i].reset_index()
    data_countries_pc[i]['x_day'] = data_countries_pc[i].index
len(data_countries_pc)

151

In [164]:
data_plot=pd.DataFrame()
for i in range(0,len(countries)):
    data_plot = pd.concat([data_plot, data_countries_pc[i]], axis=0)
# data_plot

data_plot_death_log=data_plot.copy()
def a(x):
    if x==0:
        return 0
    return np.log(x)
pd.set_option('display.max_rows', None)

data_plot['total_log_cases']=data_plot['total_cases'].apply(a)
# data_plot

In [165]:
# Plot it using Altair
source = data_plot

scales = alt.selection_interval(bind='scales', zoom=False)
selection = alt.selection_multi(fields=['location'], bind='legend')
alt.data_transformers.disable_max_rows()

base = alt.Chart(source, title = "COVID-19 Deaths Per Million of Inhabitants").encode(
    x = alt.X('date:T', title = "Days passed"),
    y = alt.Y("total_cases:Q",title = "Deaths per million"),
    color = alt.Color('location:N', legend=alt.Legend(title="Country", labelFontSize=15, titleFontSize=17),
                     scale=alt.Scale(scheme='tableau20')),
    opacity = alt.condition(selection, alt.value(1), alt.value(0.1))
)

lines = base.mark_line().add_selection(
    scales
).add_selection(
    selection
).properties(
    width=chart_width,
    height=chart_height
)

lines = base.mark_line().add_selection(
    scales
).add_selection(
    selection
).properties(
    width=chart_width,
    height=chart_height
)
lines


alt.Chart(...)

In [166]:
# base = alt.Chart(source, title = "COVID-19 Deaths Per Million of Inhabitants").encode(
#     x = alt.X('x_day:Q', title = "Days passed"),
#     y = alt.Y("total_log_cases:Q",title = "Log of deaths per million"),
#     color = alt.Color('location:N', legend=alt.Legend(title="Country", labelFontSize=15, titleFontSize=17),
#                      scale=alt.Scale(scheme='tableau20')),
#     opacity = alt.condition(selection, alt.value(1), alt.value(0.1))
# )

# lines = base.mark_line().add_selection(
#     scales
# ).add_selection(
#     selection
# ).properties(
#     width=chart_width,
#     height=chart_height
# )

# lines = base.mark_line().add_selection(
#     scales
# ).add_selection(
#     selection
# ).properties(
#     width=chart_width,
#     height=chart_height
# )
# lines